### Conhecendo a base de dados

Monta o drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Importando as bibliotecas básicas

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Carregando a Base de Dados

In [3]:
# carrega o dataset de london
from sklearn.datasets import load_boston
boston = load_boston()

In [4]:
# descrição do dataset
print (boston.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [5]:
# cria um dataframe pandas
data = pd.DataFrame(boston.data, columns=boston.feature_names)

Adicionando a coluna que será nossa variável alvo

In [6]:
# adiciona a variável MEDV
data['MEDV'] = boston.target

In [7]:
# filtra os top 16 maiores registro da coluna MEDV
top16 = data.nlargest(16, 'MEDV').index

In [8]:
# remove os valores listados em top16
data.drop(top16, inplace=True)

**Definindo um Baseline**

- `Uma baseline é importante para ter marcos no projeto`.
- `Permite uma explicação fácil para todos os envolvidos`.
- `É algo que sempre tentaremos ganhar na medida do possível`.

In [9]:
# converte os dados
data.RM = data.RM.astype(int)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 490 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     490 non-null    float64
 1   ZN       490 non-null    float64
 2   INDUS    490 non-null    float64
 3   CHAS     490 non-null    float64
 4   NOX      490 non-null    float64
 5   RM       490 non-null    int64  
 6   AGE      490 non-null    float64
 7   DIS      490 non-null    float64
 8   RAD      490 non-null    float64
 9   TAX      490 non-null    float64
 10  PTRATIO  490 non-null    float64
 11  B        490 non-null    float64
 12  LSTAT    490 non-null    float64
 13  MEDV     490 non-null    float64
dtypes: float64(13), int64(1)
memory usage: 57.4 KB


In [11]:
# estatística descritiva da coluna numero de quartos
data.RM.describe()

count    490.000000
mean       5.740816
std        0.737657
min        3.000000
25%        5.000000
50%        6.000000
75%        6.000000
max        8.000000
Name: RM, dtype: float64

In [12]:
# definindo a regra para categorizar os dados
categorias = []

In [13]:
# alimenta a lista categorias
for i in data.RM.iteritems():
  valor = (i[1])
  if valor <= 4:
    categorias.append('Pequeno')
  elif valor < 7:
    categorias.append('Medio')
  else:
    categorias.append('Grande')

In [14]:
# cria a coluna categorias
data['categorias'] = categorias

In [15]:
# imprime a contagem de categorias
data.categorias.value_counts()

Medio      424
Grande      52
Pequeno     14
Name: categorias, dtype: int64

In [16]:
# agrupa as categorias e calcula as médias
medias_categorias = data.groupby(by='categorias')['MEDV'].mean()

In [17]:
# visualizando a variável medias_categorias
medias_categorias

categorias
Grande     35.719231
Medio      20.130425
Pequeno    14.921429
Name: MEDV, dtype: float64

In [18]:
# criando o dicionario com chaves medio, grande e pequeno e seus valores
dic_baseline = {'Grande': medias_categorias[0], 'Medio': medias_categorias[1], 'Pequeno': medias_categorias[2]}

In [19]:
# imprime dicionario
dic_baseline

{'Grande': 35.71923076923077,
 'Medio': 20.1304245283019,
 'Pequeno': 14.921428571428574}

In [20]:
# cria a função retorna baseline
def retorna_baseline(num_quartos):
  if num_quartos <= 4:
    return dic_baseline.get('Pequeno')
  elif num_quartos < 7:
    return dic_baseline.get('Medio')
  else:
    return dic_baseline.get('Grande')

In [21]:
# chama a função retorna baseline
retorna_baseline(7)

35.71923076923077

In [22]:
for i in data.RM.iteritems():
  n_quartos = i[1]
  print('Número de quartos é: {} , Valor médio: {}'.format(n_quartos,retorna_baseline(n_quartos)))

Número de quartos é: 6 , Valor médio: 20.1304245283019
Número de quartos é: 6 , Valor médio: 20.1304245283019
Número de quartos é: 7 , Valor médio: 35.71923076923077
Número de quartos é: 6 , Valor médio: 20.1304245283019
Número de quartos é: 7 , Valor médio: 35.71923076923077
Número de quartos é: 6 , Valor médio: 20.1304245283019
Número de quartos é: 6 , Valor médio: 20.1304245283019
Número de quartos é: 6 , Valor médio: 20.1304245283019
Número de quartos é: 5 , Valor médio: 20.1304245283019
Número de quartos é: 6 , Valor médio: 20.1304245283019
Número de quartos é: 6 , Valor médio: 20.1304245283019
Número de quartos é: 6 , Valor médio: 20.1304245283019
Número de quartos é: 5 , Valor médio: 20.1304245283019
Número de quartos é: 5 , Valor médio: 20.1304245283019
Número de quartos é: 6 , Valor médio: 20.1304245283019
Número de quartos é: 5 , Valor médio: 20.1304245283019
Número de quartos é: 5 , Valor médio: 20.1304245283019
Número de quartos é: 5 , Valor médio: 20.1304245283019
Número d

In [23]:
# imprime as 5 primeiras linhas do dataframe
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV,categorias
0,0.00632,18.0,2.31,0.0,0.538,6,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,Medio
1,0.02731,0.0,7.07,0.0,0.469,6,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,Medio
2,0.02729,0.0,7.07,0.0,0.469,7,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,Grande
3,0.03237,0.0,2.18,0.0,0.458,6,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,Medio
4,0.06905,0.0,2.18,0.0,0.458,7,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,Grande


### Trabalhando com a baseline

Preparando os dados

In [24]:
# remove as colunas colineares, variavel alvo e a coluna categorias
X = data.drop(['RAD','TAX','MEDV','DIS','AGE','ZN','categorias'], axis=1)

In [25]:
# variável 'y' é nossa variável alvo
y = data['MEDV']

In [26]:
# Dados na variável X
X.head()

,CRIM,INDUS,CHAS,NOX,RM,PTRATIO,B,LSTAT
0,0.00632,2.31,0.0,0.538,6,15.3,396.90,4.98
1,0.02731,7.07,0.0,0.469,6,17.8,396.90,9.14
2,0.02729,7.07,0.0,0.469,7,17.8,392.83,4.03
3,0.03237,2.18,0.0,0.458,6,18.7,394.63,2.94
4,0.06905,2.18,0.0,0.458,7,18.7,396.90,5.33


In [27]:
# Variável alvo
y.head()

0    24.0
1    21.6
2    34.7
3    33.4
4    36.2
Name: MEDV, dtype: float64

Separando os conjuntos de treino e teste

In [28]:
# carrega o método train_test_split
from sklearn.model_selection import train_test_split

In [29]:
# divide os dados entre o conjunto de treino e teste, 80% e 20% respectivamente.
# define qualquer valor para o parâmetro random_state.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=5)

In [30]:
# imprime a quantidade de linhas dos conjuntos
print ('X_train: numero de linhas e colunas: {}'.format(X_train.shape))
print ('X_test: numero de linhas e colunas: {}'.format(X_test.shape))
print ('y_train: numero de linhas e colunas: {}'.format(y_train.shape))
print ('y_test: numero de linhas e colunas: {}'.format(y_test.shape))

X_train: numero de linhas e colunas: (392, 8)
X_test: numero de linhas e colunas: (98, 8)
y_train: numero de linhas e colunas: (392,)
y_test: numero de linhas e colunas: (98,)


Modelo de baseline

In [ ]:
# imprimindo as 5 primeiras linhas dos dados de teste
X_test.head()

In [31]:
# definindo uma lista de predicoes
predicoes = []

In [32]:
# para cada elemento do teste alimenta a lista predicoes
for i in X_test.RM.iteritems():
  n_quartos = i[1]
  predicoes.append(retorna_baseline(n_quartos))

In [33]:
# imprime as 10 primeiras linhas da lista de predicoes
predicoes[:10]

[20.1304245283019,
 20.1304245283019,
 20.1304245283019,
 20.1304245283019,
 20.1304245283019,
 20.1304245283019,
 20.1304245283019,
 20.1304245283019,
 35.71923076923077,
 35.71923076923077]

Atribuindo resultados a um dataframe

-   `Manteremos um log ao longo do projeto`
-   `Permite a evolução dado o baseline`



In [34]:
# cria um dataframe vazio
df_results = pd.DataFrame()

In [35]:
# adiciona a coluna valor_real
df_results['valor_real'] = y_test.values

In [36]:
# cria a coluna valor_predito_baseline com as predicoes
df_results['valor_predito_baseline'] = predicoes

In [37]:
# imprimindo as 10 primeiras linhas do df_results
df_results.head(10)

,valor_real,valor_predito_baseline
0,18.5,20.130425
1,12.7,20.130425
2,21.4,20.130425
3,23.7,20.130425
4,20.8,20.130425
5,25.0,20.130425
6,9.7,20.130425
7,14.9,20.130425
8,34.9,35.719231
9,31.6,35.719231


Plota o valor real e o baseline

In [38]:
import plotly.graph_objects as go

# Create traces
fig = go.Figure()

# Linha com os dados de teste
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_real,
                         mode='lines+markers',
                         name='Valor Real'))

# Linha com os dados preditos
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_baseline,
                         mode='lines+markers',
                         name='Valor Predito Baseline'))
# Plota a figura
fig.show()

Avaliação do modelo baseline

In [39]:
# calcula a métrica rmse
from sklearn.metrics import mean_squared_error
from math import sqrt

In [40]:
rmse = (np.sqrt(mean_squared_error(y_test, predicoes)))

In [41]:
# imprime a performance do modelo
print ('Performance do modelo baseline:')
print('\nRMSE é: {} '.format(rmse))

Performance do modelo baseline:

RMSE é: 6.205816494411828 


### Machine Learning

#### Regressão Linear

- `Devido a correlação entre algumas features podemos usar uma regressão linear.`
- `Modelo simples e eficiente.`
- `Solução rápida e robusta.`

In [42]:
# carrega o pacote LinearRegression
from sklearn.linear_model import LinearRegression

In [43]:
# cria o objeto do tipo LinearRegression
lin_model = LinearRegression()

In [44]:
# treina o algoritmo de regressão linear
lin_model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [45]:
# avaliação do modelo nos dados de teste
y_pred = lin_model.predict(X_test)

In [46]:
# calcula a métrica rmse
rmse = (np.sqrt(mean_squared_error(y_test, y_pred)))

In [47]:
# imprime a performance do modelo
print ('Performance do modelo avaliado com os dados de teste:')
print('\nRMSE é: {} '.format(rmse))

Performance do modelo avaliado com os dados de teste:

RMSE é: 4.460277295153906 


In [48]:
# atribui os resultados no dataframe df_results
df_results['valor_predito_reg_linear'] = lin_model.predict(X_test)

In [49]:
df_results.head(10)

,valor_real,valor_predito_baseline,valor_predito_reg_linear
0,18.5,20.130425,18.459170
1,12.7,20.130425,12.279894
2,21.4,20.130425,24.588307
3,23.7,20.130425,28.254693
4,20.8,20.130425,18.195439
5,25.0,20.130425,21.630648
6,9.7,20.130425,11.319198
7,14.9,20.130425,14.561032
8,34.9,35.719231,31.623717
9,31.6,35.719231,29.588133


In [50]:
import plotly.graph_objects as go

# Create traces
fig = go.Figure()

# Linha com os dados de teste
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_real,
                         mode='lines+markers',
                         name='Valor Real'))

# Linha com os dados de baseline
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_baseline,
                         mode='lines+markers',
                         name='Baseline'))

# Linha com os dados preditos pela regressão linear
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_reg_linear,
                         mode='lines',
                         line = dict(color = '#FEBFB3'),
                         name='Valor Predito Regressão Linear'))

# Plota a figura
fig.show()

##### Arvore de decisão

*   `Algoritmo de aprendizado supervisionado`
*   `Explicabilidade do Modelo`
*   `Simples`


In [51]:
# importa o pacote DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor

In [52]:
# cria um objeto do tipo DecisionTreeRegressor
regressor = DecisionTreeRegressor()

In [53]:
# treina o algoritmo
regressor.fit(X_train, y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

Avaliando o modelo de Árvore de Decisão

In [54]:
# fazendo as previsões
y_pred = regressor.predict(X_test)

In [55]:
# Adicionando o valor do modelo no dataframe df_results
df_results['valor_predito_arvore'] = y_pred

In [56]:
# visualiza o dataframe df_results
df_results.head(10)

,valor_real,valor_predito_baseline,valor_predito_reg_linear,valor_predito_arvore
0,18.5,20.130425,18.459170,17.8
1,12.7,20.130425,12.279894,23.2
2,21.4,20.130425,24.588307,24.5
3,23.7,20.130425,28.254693,23.3
4,20.8,20.130425,18.195439,21.8
5,25.0,20.130425,21.630648,27.5
6,9.7,20.130425,11.319198,7.2
7,14.9,20.130425,14.561032,17.1
8,34.9,35.719231,31.623717,36.4
9,31.6,35.719231,29.588133,31.7


In [57]:
import plotly.graph_objects as go

# cria uma figura
fig = go.Figure()

# Linha com os dados de teste
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_real,
                         mode='lines+markers',
                         name='Valor Real'))

# Linha com os dados de teste
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_baseline,
                         mode='lines+markers',
                         name='Valor Predito Baseline'))


# Linha com os dados de teste
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_reg_linear,
                         mode='lines+markers',
                         name='Valor Predito Reg Liner'))

# Linha com os dados preditos
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_arvore,
                         mode='lines+markers',
                         name='Valor Predito Arvore'))
# Plota a figura
fig.show()

Avaliação do Modelo

In [58]:
# calcula a métrica rmse
rmse = (np.sqrt(mean_squared_error(y_test, y_pred)))

In [59]:
# imprime a performance do modelo
print ('Performance do modelo avaliado com os dados de teste:')
print('\nRMSE é: {} '.format(rmse))

Performance do modelo avaliado com os dados de teste:

RMSE é: 4.88995221378095 


#### Random Forest

- `Método Ensemble`
- `Mais robustez para os modelos`

In [60]:
# Importa o método RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor

In [61]:
# cria o objeto rf_regressor
rf_regressor = RandomForestRegressor()

In [62]:
# treina o algoritmo
rf_regressor.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [63]:
# avaliação do modelo nos dados de teste
y_pred = rf_regressor.predict(X_test)

In [64]:
# adiciona os resultados no dataframe df_results
df_results['valor_predito_random_forest'] = rf_regressor.predict(X_test)

In [65]:
# calcula a métrica rmse
rmse = (np.sqrt(mean_squared_error(y_test, y_pred)))

In [66]:
# imprime a performance do modelo
print ('Performance do modelo avaliado com os dados de teste:')
print('\nRMSE é: {} '.format(rmse))

Performance do modelo avaliado com os dados de teste:

RMSE é: 3.3896804993620115 


In [67]:
# plota os resultados dos modelos e o valor real.
import plotly.graph_objects as go

# Create traces
fig = go.Figure()

# Linha com os dados de teste
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_real,
                         mode='markers',
                         name='Valor Real'))

# Linha com os dados da baseline
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_baseline,
                         mode='lines+markers',
                         line = dict(color = '#FF00FF'),
                         name='Valor da Baseline'))

# Linha com os dados preditos pela arvore
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_arvore,
                         mode='lines',
                         line = dict(color = '#B2FF66'),
                         name='Valor Predito Árvore'))

# Linha com os dados preditos pela regressão linear
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_reg_linear,
                         mode='lines',
                         line = dict(color = '#17BECF'),
                         name='Valor Predito Regressão Linear'))


# Linha com os dados preditos pela Random Forest
fig.add_trace(go.Scatter(x=df_results.index,
                         y=df_results.valor_predito_random_forest,
                         mode='lines',
                         line = dict(color = '#7F7F7F'),
                         name='Valor Predito Random Forest'))

# Plota a figura
fig.show()

### Tunning do Algoritmo de Machine Learning

**Parâmetros da Random Forest**

- `O parâmetro n_estimators define a quantidade de árvores que serão usadas para criação dos modelos. Quanto maior o número de árvores, maior será o número de modelos no ensemble.`

- `O parâmtro max_deph define a profundidade da árvore, quanto mais profunda a árvore, mais especializado nos dados de treino o nosso modelo estará.`

- `O parâmetro max_features define a quantidade máxima de nós que serão usados para separar os níveis folha da árvore`

Deploy da Aplicação

In [68]:
X['MEDV'] = y

In [69]:
X.head()

,CRIM,INDUS,CHAS,NOX,RM,PTRATIO,B,LSTAT,MEDV
0,0.00632,2.31,0.0,0.538,6,15.3,396.90,4.98,24.0
1,0.02731,7.07,0.0,0.469,6,17.8,396.90,9.14,21.6
2,0.02729,7.07,0.0,0.469,7,17.8,392.83,4.03,34.7
3,0.03237,2.18,0.0,0.458,6,18.7,394.63,2.94,33.4
4,0.06905,2.18,0.0,0.458,7,18.7,396.90,5.33,36.2


In [70]:
X.to_csv('data.csv', index=False)